In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.utils import to_categorical
import IO
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [12]:
df = IO.load_excel("./data/generated_examinations.xlsx").dropna()
category_counts = df['Diagnose'].value_counts()
# Step 2: Determine the count of the least frequent category
min_count = category_counts.min()
# Step 3: Sample the same number of entries for each category
data = pd.concat([
    df[df['Diagnose'] == category].sample(min_count, random_state=1)
    for category in category_counts.index
])
# Step 4: Reset index (optional)
data.reset_index(drop=True, inplace=True)
category_counts = data['Diagnose'].value_counts()

# Print the amount of values in each category
print("Category counts in 'Diagnosis' column:")
for category, count in category_counts.items():
    print(f"{category}: {count}")
data.head()

Category counts in 'Diagnosis' column:
Kreuzband: 173
Meniskus: 173


,Patientid_x,Text_Bef,Text_ANA,Geschlecht,Diagnose,All,Formatted,generated_text
0,307174,Beinachse gerade Rechtes Kniegelenk: Extension...,OP am 27.05.2024. Olsi Reka stellt sich heute ...,männlich,Kreuzband,beinachse gerade rechtes kniegelenk extension/...,beinachse gerade rechtes kniegelenk extension/...,Das rechte Kniegelenk hat keine Überstreckung...
1,304155,"Gangbild unter Vollbelastung flüssig, rechtes...","2 Mon nach AC Knie und MFX lat Tibiaplateau , ...",männlich,Kreuzband,"gangbild unter vollbelastung flüssig, rechtes ...","gangbild unter vollbelastung flüssig, rechtes ...",Das Gangbild ist unter voller Belastung flüss...
2,405299,Gangbild unter Vollbelastung flüssig und mit f...,2021 Arbeitsunfall mit Syndesmosenriss rechts....,weiblich,Kreuzband,"gangbild unter vollbelastung flüssig, mit fehl...","gangbild unter vollbelastung flüssig, rechtes ...",Das Gangbild unter voller Belastung ist flüss...
3,307079,Gangbild unter Vollbelastung mit fehlender Sch...,Schmerz und Entzündungsgefühl im Knie seit der...,männlich,Kreuzband,gangbild unter vollbelastung mit fehlender sch...,gangbild unter vollbelastung mit fehlender sch...,Unter voller Belastung gibt es ein Gangbild m...
4,306656,"Gangbild flüssig, Viertelkniebeuge im Einbeins...",WV bei LCA Partialinsuffizienz. Aktuell komme ...,männlich,Kreuzband,"gangbild flüssig, viertelkniebeuge im einbeins...","gangbild flüssig, knie links erguss kein","Das Gangbild ist flüssig, und es gibt keinen ..."


In [14]:
data['FullText'] = data['Text_ANA'] + ' ' + data['generated_text']
# Encode labels
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Diagnose'])
num_classes = (len(data['label'].unique()))
print(f"Number of classes: {num_classes}")
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['FullText'], data['label'], test_size=0.2, random_state=42)
train_labels = np.unique(y_train)
test_labels = np.unique(y_test)

# Tokenize the text
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
print(X_train_seq[0])


# Find the maximum sequence length
max_length_train = max(len(seq) for seq in X_train_seq)
max_length_test = max(len(seq) for seq in X_test_seq)
actual_max_length = max(max_length_train, max_length_test)
print(f"Actual maximum length: {actual_max_length}")

# Pad the sequences
max_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')
print(X_train_pad[0])

# Convert labels to categorical
y_train_cat = to_categorical(y_train)
print(y_train_cat)
y_test_cat = to_categorical(y_test)
IO.save_training_data("MenKreuzAnEx", X_train, X_test, y_train, y_test, X_train_pad, X_test_pad, y_train_cat, y_test_cat, label_encoder.classes_)


Number of classes: 2
[61, 101, 25, 33, 109, 109, 43, 81, 6, 20, 189, 44, 293, 48, 56, 60, 5, 478, 18, 224, 687, 375, 224, 2, 26, 31, 13, 37, 82, 10, 29, 1, 4, 12, 14, 1, 5, 23, 9, 2, 6, 30, 52, 1, 4, 24, 45, 35, 3, 27, 18, 62, 78]
Actual maximum length: 112
[ 61 101  25  33 109 109  43  81   6  20 189  44 293  48  56  60   5 478
  18 224 687 375 224   2  26  31  13  37  82  10  29   1   4  12  14   1
   5  23   9   2   6  30  52   1   4  24  45  35   3  27  18  62  78   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [